<a href="https://colab.research.google.com/github/jyotidabass/Binary-text-classifier-ham_spam-/blob/main/Binary_health_classifier(ham_spam).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/sudeshmu/binary_text_classifier.git

Cloning into 'binary_text_classifier'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 24 (delta 5), reused 14 (delta 2), pack-reused 0
Unpacking objects: 100% (24/24), done.


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
"""
import pandas as pd
import pickle as pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Embedding, Input, Activation, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from keras.models import Model
from keras.callbacks import EarlyStopping

dataset_file_path = "/content/binary_text_classifier/spam.csv"

max_words = 1000
max_len = 150
train_test_ratio = 0.8
train_batch_size=128
n_epochs=10

def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

def train_n_test ():
    data = pd.read_csv (dataset_file_path, delimiter=',', encoding='latin-1')
    X = data.v2
    y = data.v1
    
    label_encoder = LabelEncoder ()
    y = label_encoder.fit_transform (y)
    
    X_train, X_test, y_train, y_test = train_test_split (X, y, 
                                                         train_size=train_test_ratio, 
                                                         shuffle=True)
    tok = Tokenizer(num_words=max_words)
    tok.fit_on_texts(X_train)
    sequences = tok.texts_to_sequences(X_train)
    sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
    
    model = RNN()
    model.summary()
    model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])
    
    model.fit(sequences_matrix,y_train,batch_size=train_batch_size,epochs=n_epochs,
              validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])
    
    print ("Training Completed")
    
    test_sequences = tok.texts_to_sequences(X_test)
    test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
    
    accr = model.evaluate(test_sequences_matrix,y_test)
    print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
    

train_n_test ()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   